In [ ]:
!pip install trl -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.


In [ ]:
!pip install wandb -q

In [ ]:
!pip install -q pytorch_warmup

In [ ]:
from datasets import load_dataset
import numpy as np
from datasets import Dataset
from trl import RewardTrainer, RewardConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer, GPT2LMHeadModel
import random
import torch
import torch.nn.functional as F

import collections
from tqdm import tqdm
from copy import deepcopy
import pytorch_warmup as warmup

data_imdb = load_dataset("stanfordnlp/imdb")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SAVE_REWARD_DIR = ""
WARP_SAVE_DIR = ""

2024-08-05 17:40:09.442299: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 17:40:09.442413: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 17:40:09.563439: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
def build_dataset(tokenizer, input_size, train=True):
    if train:
        data_generator = zip(data_imdb['train'][len(data_imdb['train']) // 2:]['text'],
                             data_imdb['train'][:len(data_imdb['train']) // 2]['text'])
    else:
        data_generator = zip(data_imdb['test'][-100:]['text'],
                             data_imdb['test'][:100]['text'])
    def gen():
        for x in data_generator:
            yield {"chosen": x[0], "rejected": x[1]}
    pairs = Dataset.from_generator(gen)

    def tokenize(sample):
        dict_chosen = tokenizer(sample["chosen"])
        sample["input_ids_chosen"] = dict_chosen["input_ids"][: input_size]
        sample["attention_mask_chosen"] = dict_chosen["attention_mask"][: input_size]

        dict_rejected = tokenizer(sample["rejected"])
        sample["input_ids_rejected"] = dict_rejected["input_ids"][: input_size]
        sample["attention_mask_rejected"] = dict_rejected["attention_mask"][: input_size]
        return sample
    ds = pairs.map(tokenize)
    return ds

# Training reward model

In [ ]:
base_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased")
base_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
reward_config = RewardConfig(output_dir="model", num_train_epochs=5, learning_rate=1e-5)

base_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
base_model.resize_token_embeddings(len(base_tokenizer))

In [ ]:
input_size = 7
ds = build_dataset(base_tokenizer, input_size)

In [ ]:
trainer = RewardTrainer(model=base_model,
                        tokenizer=base_tokenizer,
                        train_dataset=ds,
                        args=reward_config)

trainer.train()

In [ ]:
trainer.save_model("SAVE_REWARD_DIR")
!zip -r SAVE_REWARD_DIR.zip SAVE_REWARD_DIR

# Training WARP

In [ ]:
reward_model = AutoModelForSequenceClassification.from_pretrained(SAVE_REWARD_DIR, local_files_only=True).to(device)
sft_model = GPT2LMHeadModel.from_pretrained("lvwerra/gpt2-imdb").to(device)
sft_tokenizer = AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb", padding_side="left")
anchor_model = GPT2LMHeadModel.from_pretrained("lvwerra/gpt2-imdb").to(device)

config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:
sft_tokenizer.pad_token_id = sft_tokenizer.eos_token_id
sft_model.resize_token_embeddings(len(sft_tokenizer))
sft_model.config.pad_token_id = sft_tokenizer.pad_token_id

reward_model.resize_token_embeddings(len(sft_tokenizer))
reward_model.config.pad_token_id = sft_tokenizer.pad_token_id

anchor_model.resize_token_embeddings(len(sft_tokenizer))
anchor_model.config.pad_token_id = sft_tokenizer.pad_token_id

In [ ]:
import wandb
wandb.login()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
def collator(data):
    batches = []
    for i in range(0, len(data), batch_size):
        cur_data = data[i:min(len(data), i + batch_size)]
        batch = {}
        batch['input_ids'] = torch.tensor(cur_data['input_ids_chosen'] + cur_data['input_ids_rejected'])
        batch['attention_mask'] = torch.tensor(cur_data['attention_mask_chosen'] + cur_data['attention_mask_rejected'])
        batches.append(batch)
    return batches

In [ ]:
input_size = 7
ds = build_dataset(sft_tokenizer, input_size)

batch_size = 32
ds_batched = collator(ds)
ds_batched[0]['input_ids'].shape

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors


torch.Size([64, 7])

In [ ]:
def adjusted_kl_loss(y_ema, y_m):
    log_ym = torch.log(y_m)
    log_yma = torch.log(y_ema)

    kl_loss = F.kl_div(log_ym, log_yma, log_target=True, reduction="batchmean")
    reward = reward_model(y_m.argmax(axis=2)).logits.mean() - beta * kl_loss
    return -reward

In [ ]:
def update_theta_ema(theta_ema, theta_m):
    theta_ema_new = {}
    for key in theta_ema.keys():
        theta_ema_new[key] = (1 - mu) * theta_ema[key] + mu * theta_m[key]
    return theta_ema_new

In [ ]:
def inner_cycle(theta_init, run_name):
    '''
    training cycle from the paper
    '''
    run = wandb.init(project="second_try", name=run_name, reinit=True)
    cfg = wandb.config
    cfg.update({"T" : T, "batch_size": batch_size, "lr" : 1e-6,
                "M": M, "I": I, "beta": beta, "mu": mu, "eta": eta, "warmup": 10})
    anchor_model.load_state_dict(theta_init)
    sft_model.load_state_dict(theta_init)
    theta_m, theta_ema = sft_model.state_dict(), anchor_model.state_dict()

    optimizer = torch.optim.Adam(sft_model.parameters(), lr=1e-6)
    mean_kl_loss = 0
    for t in tqdm(range(T), desc="Training: "):
        for i, batch in enumerate(ds_batched):
            optimizer.zero_grad()
            input_ids, attention_mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)
            logits_m = sft_model(input_ids=input_ids,
                                 attention_mask=attention_mask).logits[:, :, input_size-1:-1]
            logits_ema = anchor_model(input_ids=input_ids,
                                    attention_mask=attention_mask).logits[:, :, input_size-1:-1]

            ema_dist = torch.clamp(F.softmax(logits_ema, dim=2), min=1e-9, max=1.)
            m_dist = torch.clamp(F.softmax(logits_m, dim=2), min=1e-9, max=1.)
            kl_loss = adjusted_kl_loss(ema_dist, m_dist)
            mean_kl_loss += kl_loss.item()

            kl_loss.backward()
            optimizer.step()

            theta_ema = update_theta_ema(theta_m, theta_ema)
        wandb.log({"reward" : abs(mean_kl_loss) / len(ds_batched)})
    run.finish()
    return theta_m

In [ ]:
theta_init = deepcopy(sft_model.state_dict())
I = 2
M = 2
T = 10
beta = 0.1
mu = 0.01
eta = 0.5

In [ ]:
def get_task_vector(pretrained_state_dict, finetuned_state_dict):
    with torch.no_grad():
        vector = {}
        for key in pretrained_state_dict.keys():
            vector[key] = finetuned_state_dict[key] - pretrained_state_dict[key]
    return vector

In [ ]:
def slerp(theta_init, theta_1, theta_2, lambda_):
    def find_angle(v1, v2):
        dot_product = (torch.flatten(v1) * torch.flatten(v2)).sum()
        cos_angle = torch.clamp((dot_product / (torch.linalg.vector_norm(v1) * torch.linalg.vector_norm(v2))), min=-1., max=1.).item()
        return np.arccos(cos_angle), np.arccos(cos_angle) * 180 / torch.pi

    delta_1 = get_task_vector(theta_init, theta_1)
    delta_2 = get_task_vector(theta_init, theta_2)

    theta_fin = {}
    for key in delta_1.keys():
        omega_radian, omega_degrees = find_angle(delta_1[key], delta_2[key])
        if abs(np.sin(omega_radian)) > 1e-7:
            theta_fin[key] = theta_init[key] \
            + np.sin((1 - lambda_) * omega_degrees) / np.sin(omega_radian) * delta_1[key] \
            + np.sin(lambda_ * omega_degrees) / np.sin(omega_radian) * delta_2[key]
        else:
            theta_fin[key] = theta_init[key]
    return theta_fin


In [ ]:
def warp(theta_init):
    def update_theta_init(theta_init, theta_slerp, eta):
        for key in theta_init.keys():
            theta_init[key] = (1 - eta) * theta_init[key] + eta * theta_slerp[key]
        return theta_init

    for i in range(I):
        theta_1 = inner_cycle(theta_init, "first_run")
        torch.save(theta_1, f"theta1_{i}.pth")

        theta_2 = inner_cycle(theta_init, "second_run")
        torch.save(theta_2, f"theta2_{i}.pth")

        theta_slerp = slerp(theta_init, theta_1, theta_2, 0.5)
        theta_init = update_theta_init(theta_init, theta_slerp, eta)
    return theta_init

In [ ]:
theta_fin = warp(theta_init)

wandb: Currently logged in as: ntyazh (ntyazh-team). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.4
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240805_174138-g2a16csy
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run first_run
wandb: ⭐️ View project at https://wandb.ai/ntyazh-team/second_try
wandb: 🚀 View run at https://wandb.ai/ntyazh-team/second_try/runs/g2a16csy
Training: 100%|██████████| 10/10 [13:20<00:00, 80.04s/it]
wandb:                                                                                
wandb: 
wandb: Run history:
wandb: reward ▁▂▃▃▄▅▆▆▇█
wandb: 
wandb: Run summary:
wandb: reward 0.43794
wandb: 
wandb: 🚀 View run first_run at: https://wandb.ai/ntyazh-team/second_try/runs/g2a16csy
wandb: ⭐️ View project at: https://wandb.ai/ntyazh-team/second_try
wandb: Synced 5 W&B file(s), 0 media 

In [ ]:
torch.save(theta_fin, f"{WARP_SAVE_DIR}.pth")

# SFT-model VS trained WARP-model

In [ ]:
sft_model = GPT2LMHeadModel.from_pretrained("lvwerra/gpt2-imdb").to(device)
sft_tokenizer = AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb", padding_side="left")

In [ ]:
sft_tokenizer.pad_token_id = sft_tokenizer.eos_token_id
sft_model.resize_token_embeddings(len(sft_tokenizer))
sft_model.config.pad_token_id = sft_tokenizer.pad_token_id

In [ ]:
warp_model = GPT2LMHeadModel.from_pretrained("lvwerra/gpt2-imdb").to(device)
warp_model.load_state_dict(torch.load(WARP_SAVE_DIR))
warp_model.resize_token_embeddings(len(sft_tokenizer))
warp_model.config.pad_token_id = sft_tokenizer.pad_token_id

In [ ]:
input_size = 7
ds = build_dataset(sft_tokenizer, input_size, train=False)

batch_size = 32
ds_batched = collator(ds)
ds_batched[0]['input_ids'].shape

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1175 > 1024). Running this sequence through the model will result in indexing errors


torch.Size([64, 7])

In [ ]:
mean_kl_sft = 0
mean_kl_warp = 0
mean_reward_sft, mean_reward_warp = 0, 0
with torch.no_grad():
    for i, batch in enumerate(ds_batched):
        input_ids, attention_mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)
        logits_sft = sft_model(input_ids=input_ids,
                                             attention_mask=attention_mask).logits[:, :, input_size-1:-1]
        logits_warp = warp_model(input_ids=input_ids,
                                 attention_mask=attention_mask).logits[:, :, input_size-1:-1]


        sft_dist = torch.clamp(F.softmax(logits_sft, dim=2), min=1e-9, max=1.)
        warp_dist = torch.clamp(F.softmax(logits_warp, dim=2), min=1e-9, max=1.)
        log_sft_dist = torch.log(sft_dist)
        log_warp_dist = torch.log(warp_dist)

        kl_loss_sft = F.kl_div(log_sft_dist, log_warp_dist, log_target=True, reduction="batchmean")
        kl_loss_warp = F.kl_div(log_warp_dist, log_sft_dist, log_target=True, reduction="batchmean")
        reward_sft = reward_model(sft_dist.argmax(axis=2)).logits.mean()
        reward_warp = reward_model(warp_dist.argmax(axis=2)).logits.mean()

        mean_kl_sft += kl_loss_sft
        mean_kl_warp += kl_loss_warp
        mean_reward_sft += reward_sft
        mean_reward_warp += reward_warp

print(f"Mean SFT reward: {mean_reward_sft / len(ds_batched):.7f}\nMean WARP reward: {mean_reward_warp / len(ds_batched):.7f}\n")
print(f"Mean KL(sft||warp): {mean_kl_sft / len(ds_batched):.5f}\nMean KL(warp||sft): {mean_kl_warp / len(ds_batched):.5f} ")

Mean SFT reward: 0.0438411
Mean WARP reward: 0.0438410

Mean KL(sft||warp): 0.02546
Mean KL(warp||sft): 0.02410 
